# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID               City      Lat       Lng  Max Temp  Humidity  \
0        0          papatowai -46.5619  169.4708     10.71        98   
1        1  port-aux-francais -49.3500   70.2167     -0.40        71   
2        2           waitangi -43.9535 -176.5597     13.79        88   
3        3               ebon   4.5820  168.6964     28.40        77   
4        4    remire-montjoly   4.9167  -52.2667     28.02        83   

   Cloudiness  Wind Speed Country        Date  
0         100        3.33      NZ  1695248768  
1         100       17.64      TF  1695248768  
2           5        4.02      NZ  1695248769  
3          91        4.94      MH  1695248769  
4          75        1.03      GF  1695248770

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    scale =1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df[(city_data_df["Max Temp"]<27) & (city_data_df["Max Temp"]>21)&(city_data_df["Wind Speed"]<4.5)& (city_data_df["Cloudiness"]==0)]

narrowed_city_df

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

City_ID             City      Lat       Lng  Max Temp  Humidity  \
42        42          toliara -23.3500   43.6667     21.38        81   
64        64      farafangana -22.8167   47.8333     21.33        90   
73        73           bethel  41.3712  -73.4140     22.47        69   
89        89        ar rutbah  33.0381   40.2848     26.23        18   
123      123         vallenar -28.5708  -70.7581     22.51        32   
161      161        quelimane -17.8786   36.8883     22.97        94   
224      224      kapuskasing  49.4169  -82.4331     21.05        37   
253      253         florence  43.7667   11.2500     21.33        72   
259      259          awjilah  29.1081   21.2869     24.99        36   
319      319       rapid city  44.0805 -103.2310     24.61        38   
335      335            beira -19.8436   34.8389     22.05       100   
339      339      tamanrasset  22.7850    5.5228     23.95        17   
356      356            lucas  41.6501  -83.6666     26.32        53   
370      370    villa dolores -31.9459  -65.1896     23.71        37   
461      461          dauphin  51.1494 -100.0502     22.08        30   
474      474      al kharijah  25.4514   30.5464     26.60        35   
492      492     saint-pierre -21.3393   55.4781     21.98        73   
520      520     port alberni  49.2413 -124.8028     23.29        49   
588      588  cruzeiro do sul -29.5128  -51.9853     24.17        94   

     Cloudiness  Wind Speed Country        Date  
42            0        2.13      MG  1695248792  
64            0        3.23      MG  1695248806  
73            0        1.54      US  1695248811  
89            0        4.30      IQ  1695248820  
123           0        2.94      CL  1695248842  
161           0        1.03      MZ  1695248870  
224           0        3.60      CA  1695248890  
253           0        0.51      IT  1695248924  
259           0        3.84      LY  1695248927  
319           0        3.60      US  1695248674  
335           0        2.57      MZ  1695248973  
339           0        2.06      DZ  1695248975  
356           0        3.09      US  1695248983  
370           0        2.89      AR  1695248991  
461           0        2.57      CA  1695249092  
474           0        3.88      EG  1695249099  
492           0        2.06      RE  1695249109  
520           0        1.34      CA  1695249128  
588           0        1.75      BR  1695249176

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

City Country      Lat       Lng  Humidity Hotel Name
42           toliara      MG -23.3500   43.6667        81           
64       farafangana      MG -22.8167   47.8333        90           
73            bethel      US  41.3712  -73.4140        69           
89         ar rutbah      IQ  33.0381   40.2848        18           
123         vallenar      CL -28.5708  -70.7581        32           
161        quelimane      MZ -17.8786   36.8883        94           
224      kapuskasing      CA  49.4169  -82.4331        37           
253         florence      IT  43.7667   11.2500        72           
259          awjilah      LY  29.1081   21.2869        36           
319       rapid city      US  44.0805 -103.2310        38           
335            beira      MZ -19.8436   34.8389       100           
339      tamanrasset      DZ  22.7850    5.5228        17           
356            lucas      US  41.6501  -83.6666        53           
370    villa dolores      AR -31.9459  -65.1896        37           
461          dauphin      CA  51.1494 -100.0502        30           
474      al kharijah      EG  25.4514   30.5464        35           
492     saint-pierre      RE -21.3393   55.4781        73           
520     port alberni      CA  49.2413 -124.8028        49           
588  cruzeiro do sul      BR -29.5128  -51.9853        94

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"]=f"circle:{longitude},{latitude},{radius}"
    params["bias"]=f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
toliara - nearest hotel: Ambary
farafangana - nearest hotel: Le Concombre Masqué
bethel - nearest hotel: Hampton Inn Danbury
ar rutbah - nearest hotel: No hotel found
vallenar - nearest hotel: Hotel del Marqués
quelimane - nearest hotel: Mabassa
kapuskasing - nearest hotel: Travelodge
florence - nearest hotel: Old bridge Apartment
awjilah - nearest hotel: No hotel found
rapid city - nearest hotel: The Rushmore Hotel & Suites
beira - nearest hotel: Hotel Savoy
tamanrasset - nearest hotel: فندق الأمان
lucas - nearest hotel: Hawthorn Suites by Wyndham Holland/Toledo Area
villa dolores - nearest hotel: Plaza Hotel
dauphin - nearest hotel: No hotel found
al kharijah - nearest hotel: Kharga Hotel
saint-pierre - nearest hotel: Tropic Hotel
port alberni - nearest hotel: Hospitality Inn
cruzeiro do sul - nearest hotel: Estrela Palace Hotel


City Country      Lat       Lng  Humidity  \
42           toliara      MG -23.3500   43.6667        81   
64       farafangana      MG -22.8167   47.8333        90   
73            bethel      US  41.3712  -73.4140        69   
89         ar rutbah      IQ  33.0381   40.2848        18   
123         vallenar      CL -28.5708  -70.7581        32   
161        quelimane      MZ -17.8786   36.8883        94   
224      kapuskasing      CA  49.4169  -82.4331        37   
253         florence      IT  43.7667   11.2500        72   
259          awjilah      LY  29.1081   21.2869        36   
319       rapid city      US  44.0805 -103.2310        38   
335            beira      MZ -19.8436   34.8389       100   
339      tamanrasset      DZ  22.7850    5.5228        17   
356            lucas      US  41.6501  -83.6666        53   
370    villa dolores      AR -31.9459  -65.1896        37   
461          dauphin      CA  51.1494 -100.0502        30   
474      al kharijah      EG  25.4514   30.5464        35   
492     saint-pierre      RE -21.3393   55.4781        73   
520     port alberni      CA  49.2413 -124.8028        49   
588  cruzeiro do sul      BR -29.5128  -51.9853        94   

                                         Hotel Name  
42                                           Ambary  
64                              Le Concombre Masqué  
73                              Hampton Inn Danbury  
89                                   No hotel found  
123                               Hotel del Marqués  
161                                         Mabassa  
224                                      Travelodge  
253                            Old bridge Apartment  
259                                  No hotel found  
319                     The Rushmore Hotel & Suites  
335                                     Hotel Savoy  
339                                     فندق الأمان  
356  Hawthorn Suites by Wyndham Holland/Toledo Area  
370                                     Plaza Hotel  
461                                  No hotel found  
474                                    Kharga Hotel  
492                                    Tropic Hotel  
520                                 Hospitality Inn  
588                            Estrela Palace Hotel

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=1,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)